<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/tfq_hybrid_model_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq==1.0.0
!pip install tensorflow_quantum

In [ ]:
#!/usr/bin/env python3
"""
tfq_hybrid_model.py

Builds and trains a simple quantum-classical hybrid model
using TensorFlow Quantum (TFQ) and Cirq.
"""

import cirq
import sympy
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq
from tensorflow.keras import layers, models

def create_quantum_circuit():
    # Define two qubits on a grid
    qubits = [cirq.GridQubit(0, i) for i in range(2)]

    # Symbolic parameters for tunable rotation gates
    theta = sympy.symbols('theta0:2')

    # Build parameterized circuit
    circuit = cirq.Circuit(
        cirq.rx(theta[0])(qubits[0]),
        cirq.ry(theta[1])(qubits[1]),
        cirq.CNOT(qubits[0], qubits[1])
    )

    # Readout operators (Pauli Z on each qubit)
    readouts = [cirq.Z(qubit) for qubit in qubits]
    return circuit, readouts

def prepare_data(num_samples=100):
    # Generate empty circuits (data encoding can go here)
    circuits = [cirq.Circuit() for _ in range(num_samples)]

    # Binary labels (random for demo purposes)
    labels = np.random.randint(0, 2, size=(num_samples, 1)).astype(np.float32)

    # Convert Cirq circuits to TFQ tensors
    tfq_tensor = tfq.convert_to_tensor(circuits)
    return tfq_tensor, labels

def build_hybrid_model(circuit, readouts):
    # Quantum layer: parameterized circuit + expectation measurement
    quantum_layer = tfq.layers.PQC(circuit, readouts)

    # Classical post-processing layers
    model = models.Sequential([
        layers.Input(shape=(), dtype=tf.string),
        quantum_layer,                 # Outputs shape=(None, 2)
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.01),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

def main():
    # 1. Build circuit and readouts
    circuit, readouts = create_quantum_circuit()

    # 2. Prepare toy data
    x_data, y_data = prepare_data(num_samples=200)
    split = int(0.8 * len(x_data))
    x_train, x_test = x_data[:split], x_data[split:]
    y_train, y_test = y_data[:split], y_data[split:]

    # 3. Build and summarize the hybrid model
    model = build_hybrid_model(circuit, readouts)
    model.summary()

    # 4. Train the model
    model.fit(
        x_train, y_train,
        epochs=20,
        batch_size=16,
        validation_data=(x_test, y_test),
        verbose=2
    )

    # 5. Evaluate on test set
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    print(f'\nTest Loss: {loss:.4f} │ Test Accuracy: {acc:.4f}')

    # 6. Inspect raw quantum layer outputs
    quantum_layer = model.layers[1]
    sample_out = quantum_layer(x_test[:5])
    print('\nQuantum Layer Outputs:\n', sample_out.numpy())

if __name__ == '__main__':
    main()